In [161]:
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
import os
from tqdm import tqdm
import zipfile
import shutil

In [162]:
"""Распаковываем архив с заданием"""

task_arch = 'Задание2.zip'
task = zipfile.ZipFile(task_arch, 'r')
path_name = 'Задание2'

list_files=list()
for name in task.namelist():
    dec_name = name.encode('cp437').decode('utf-8')
    if dec_name.startswith(path_name):
        list_files.append(name)
        task.extract(name)
        os.rename(name, dec_name)

shutil.rmtree(list_files[0], ignore_errors=False, onerror=None)

In [163]:
"""Подготовка к обработке xml файла"""

tree = ET.parse('Задание2/masks.xml')
root = tree.getroot()

"""Директории для результатов выполнения"""

result_masks_directory = 'result/masks/'
result_photos_directory = 'result/images/'

In [164]:
image_list = []
for elem in root.findall('image'):
    image_list.append(elem)

for i in tqdm(range(len(image_list))):
    picture = f'Задание2/images/{image_list[i].get("name").split("/")[-1]}'

    """Создаем маски"""

    with Image.open(picture) as mask_img:
        mask_img.load()
        mask_img = Image.new('RGB', mask_img.size, (0, 0, 0))
        draw = ImageDraw.Draw(mask_img)
        for item in range(len(image_list[i])):
            subelem = image_list[i][item]
            if subelem.get('label') != 'Ignore':
                points = subelem.get('points').split(';')
                points = [(float(x), float(y)) for x, y in (point.split(',') for point in points)]
                draw.polygon(points, fill='magenta')
            else:
                points = subelem.get('points').split(';')
                points = [(float(x), float(y)) for x, y in (point.split(',') for point in points)]
                draw.polygon(points, fill='black')
            if not os.path.exists(result_masks_directory):
                os.makedirs(result_masks_directory)
            mask_img.save(f'{result_masks_directory}mask_{image_list[i].get("name").split("/")[-1]}.png')

    """Добавляем маски на оригинал фото"""

    with Image.open(picture) as photo:
        photo.load()
        mask_img = mask_img.convert('RGBA')
        black_pixels = mask_img.load()
        width, height = mask_img.size
        for y in range(height):
            for x in range(width):
                if black_pixels[x, y] == (0, 0, 0, 255):
                    black_pixels[x, y] = (0, 0, 0, 0)
        photo.paste(mask_img, (0, 0), mask_img)
        if not os.path.exists(result_photos_directory):
            os.makedirs(result_photos_directory)
        photo.save(f'{result_photos_directory}photo_{image_list[i].get("name").split("/")[-1]}.png')

100%|██████████| 8/8 [00:09<00:00,  1.14s/it]


In [165]:
"""Добавляем результат в архив"""

result = zipfile.ZipFile('result.zip', 'w')
for root, dirs, files in os.walk('result'):
    for file in files:
        result.write(os.path.join(root, file))
result.close()

In [166]:
"""Удаляем созданные для работы директории"""

shutil.rmtree('result', ignore_errors=False, onerror=None)
shutil.rmtree('Задание2', ignore_errors=False, onerror=None)